In [19]:
import pandas as pd
import nltk
import re
import networkx as nx
from collections import defaultdict
import random
from gensim.models import Word2Vec
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import requests

# Descargar recursos de NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer

[nltk_data] Downloading package punkt to /home/ymamani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ymamani/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/ymamani/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [20]:
df = pd.read_csv('imdb_ds_2k_clean.csv')  # ruta del dataset
#df = df[:1]

In [21]:
df

,sw_text,sentiment
0,one reviewer ha mention watch oz episode youll...,positive
1,wonderful little production film technique una...,positive
2,think wa wonderful way spend time hot summer w...,positive
3,basically family little boy jake think zombie ...,negative
4,petter matteis love time money visually stun f...,positive
...,...,...
1995,feel minnesota direct steven baigelmann star k...,negative
1996,cell rat cell like antz must watch twice appre...,positive
1997,movie despite list list celebs complete waste ...,negative
1998,love movie wa could break tear watch really up...,positive


In [22]:
df_synonyms = pd.read_csv('graph_conceptnet_dic_synonym_limit5_imdb2k_full.csv', sep=";") 
df_synonyms

,Original,Conceptos
0,godyou,NaN
1,govt,government
2,whyits,NaN
3,modestly,modest
4,moneywhatever,NaN
...,...,...
25011,betternow,NaN
25012,chronological,"time,chronograph,chronology,chronologically"
25013,demonize,"vilify,demon,demonology,represent,demonic"
25014,ministry,"minister,cabinet,portfolio,secretary-of-state,..."


In [23]:
# Inicializar lematizador, stop words y VADER
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))


In [24]:
# Función de preprocesamiento
def preprocess_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar URLs
    text = re.sub(r'http\S+|www.\S+', '', text)
    # Eliminar caracteres especiales y números
    text = re.sub(r'[^a-z\s]', '', text)
    # Tokenización
    tokens = word_tokenize(text)
    # Eliminar stop words y lematizar
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return tokens

# Preprocesar el texto y almacenar los tokens
df['tokens'] = df['sw_text'].apply(preprocess_text)

In [25]:

# Obtener el lexicón de VADER
sia = SentimentIntensityAnalyzer()
vader_lexicon = sia.lexicon  # Es un diccionario {palabra: puntuación}

# Definir umbrales para palabras positivas y negativas
positive_threshold = 2.0
negative_threshold = -2.0

# Inicializar conjuntos de palabras positivas y negativas
positive_words = set()
negative_words = set()

# Iterar sobre el lexicón y separar palabras según su puntuación
for word, score in vader_lexicon.items():
    if score >= positive_threshold:
        positive_words.add(word)
    elif score <= negative_threshold:
        negative_words.add(word)

# Construir word_polarity y all_words antes de la paralelización
word_polarity = {}
all_words = set()

for word in positive_words:
    word_polarity[word] = 1
    all_words.add(word)

for word in negative_words:
    word_polarity[word] = -1
    all_words.add(word)

# Recopilar todas las palabras de los documentos
for tokens in df['tokens']:
    for token in tokens:
        all_words.add(token)
        if token not in word_polarity:
            word_polarity[token] = 0  # Asignar polaridad neutral

In [26]:
import multiprocessing
from multiprocessing import Pool
from functools import partial
import networkx as nx
import pandas as pd

# Supongamos que ya tienes tus DataFrames `df_imdb` y `df_synonym`

# Definir la función para obtener sinónimos desde el DataFrame `df_synonym`
def get_synonyms_from_df(word, df_synonym):
    synonyms = set()
    # Filtrar el DataFrame para encontrar la fila correspondiente a la palabra
    synonyms_row = df_synonym[df_synonym['Original'] == word]['Conceptos']
    if not synonyms_row.empty:
        conceptos = synonyms_row.values[0]
        if isinstance(conceptos, str) and conceptos.strip():
            # Dividir los conceptos por comas y limpiar espacios
            synonyms.update([syn.strip().lower() for syn in conceptos.split(',')])
    return synonyms

# Definir la función de procesamiento
def process_document(tokens, df_synonym, all_words):
    nodes = set()
    edges = []
    window_size = 2  # Puedes ajustar el tamaño de la ventana si lo deseas

    for i, token in enumerate(tokens):
        nodes.add(token)
        # Conectar con palabras dentro de la ventana de contexto
        for j in range(max(0, i - window_size), min(len(tokens), i + window_size + 1)):
            if i != j:
                edges.append((token, tokens[j]))
        
        # Obtener sinónimos desde el DataFrame `df_synonym`
        synonyms = get_synonyms_from_df(token, df_synonym)
        
        
        # Añadir aristas para los sinónimos
        for synonym in synonyms:
            #print(synonym)
            #if synonym in all_words:
            edges.append((token, synonym))
            #print(token, "----", synonym)
    return nodes, edges

In [27]:
# Preparar los datos
# Suponiendo que la columna 'tokens' de `df_imdb` contiene listas de tokens por documento
tokens_list = df['tokens'].tolist()
# Asegurarse de que `all_words` sea un conjunto de todas las palabras únicas en los tokens
all_words = set(word for tokens in tokens_list for word in tokens)
# Número de procesos
n_processes = multiprocessing.cpu_count()

# Usar functools.partial para fijar `df_synonym` y `all_words`
process_document_partial = partial(process_document, df_synonym=df_synonyms, all_words=all_words)

# Crear un pool de procesos y mapear la función
with Pool(processes=n_processes) as pool:
    results = pool.map(process_document_partial, tokens_list)

# Recopilar nodos y aristas de todos los procesos
all_nodes = set()
all_edges = set()

for nodes, edges in results:
    all_nodes.update(nodes)
    all_edges.update(edges)

# Construir el grafo
G = nx.Graph()
G.add_nodes_from(all_nodes)
G.add_edges_from(all_edges)

In [28]:
print("Numero de nodos: ", G.number_of_nodes())
print("Numero de aristas: ", G.number_of_edges())

Numero de nodos:  45747
Numero de aristas:  400077


In [29]:
# Parámetros para las caminatas
num_walks_per_node = 300
walk_length = 40
alpha = 0.9  # Sesgo hacia la misma polaridad

In [30]:
# Definir la función de caminata aleatoria sesgada
def sentiment_biased_walk(start_node):
    walk = [start_node]
    for _ in range(walk_length - 1):
        curr_node = walk[-1]
        neighbors = list(G.neighbors(curr_node))
        if not neighbors:
            break
        probabilities = []
        for neighbor in neighbors:
            # Calcular polaridad
            curr_polarity = word_polarity.get(curr_node, 0)
           
            neighbor_polarity = word_polarity.get(neighbor, 0)
            
            # Si las polaridades coinciden y no son neutras, aumentar probabilidad
            if curr_polarity == neighbor_polarity and curr_polarity != 0:
                prob = alpha
                #print("curr_polarity ", curr_polarity, "-", curr_node)
                #print("neighbor_polarity ", neighbor_polarity, "-", neighbor)
                #print("proba = ",  prob)
            else:                
                prob = 1 - alpha
                #print("else proba = ",  prob)
            probabilities.append(prob)
        # Normalizar probabilidades
        total = sum(probabilities)
        #print("total sum prob = ",  total)
        probabilities = [prob / total for prob in probabilities]
        # Seleccionar siguiente nodo basado en las probabilidades
        next_node = random.choices(neighbors, weights=probabilities, k=1)[0]
        walk.append(next_node)
    return walk

# Función auxiliar para generar caminatas para un nodo
def generate_walks_for_node(node):
    node_walks = []
    for _ in range(num_walks_per_node):
        walk = sentiment_biased_walk(node)
        node_walks.append(walk)
    return node_walks

In [ ]:
# Lista de nodos
nodes = list(G.nodes())

# Generar caminatas en paralelo
with Pool(processes=n_processes) as pool:
    results = pool.map(generate_walks_for_node, nodes)

# Combinar las caminatas
walks = []
for node_walks in results:
    walks.extend(node_walks)

#45m 50.4s

In [ ]:
# Entrenar Word2Vec
walks_str = [[str(node) for node in walk] for walk in walks]  # Convertir nodos a cadenas
model = Word2Vec(walks_str, vector_size=300, window=5, min_count=1, sg=1, workers=30)
#34m 3.5s

In [ ]:
print("Guardando embeddings en archivo txt...")
#model.wv.save_word2vec_format("graph_model_embedding_cn_w5d300_le40nw300a09_2k_092025.txt", binary=False)

Guardando embeddings en archivo txt...


In [34]:
# Obtener embeddings de palabras
def get_word_embedding(word):
    if word in model.wv:
        return model.wv[word]
    else:
        return np.zeros(model.vector_size)

# Representar documentos promediando los embeddings de sus palabras
def document_embedding(tokens):
    embeddings = [get_word_embedding(token) for token in tokens]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(model.vector_size)

# Obtener embeddings de los documentos
df['embedding'] = df['tokens'].apply(document_embedding)


In [35]:
# Preparar datos para entrenamiento
X = np.stack(df['embedding'].values)
y = df['sentiment'].map({'positive': 1, 'negative': 0}).values

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [50]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import (
    accuracy_score,
    accuracy_score,
    roc_auc_score,
    f1_score,
    classification_report,
    precision_score,
    recall_score,
)

def Evaluation(model, X_train, X_test, y_train, y_test, hypertuning=False):
    
    y_pred = model.predict(X_train)
    y_pred_proba = model.predict_proba(X_train)

    accuracy_train = accuracy_score(y_train, y_pred)
    precision_train = precision_score(y_train, y_pred)
    recall_train = recall_score(y_train, y_pred)
    F1_score_train = f1_score(y_train, y_pred)
    # print("F1_Score = ", F1_score_train )
    roc_auc_train = roc_auc_score(y_train, y_pred_proba[:, 1])
    # print( classification_report( y_train, y_pred ) )

    # print( " For Test Set :  ")
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)

    accuracy_test = accuracy_score(y_test, y_pred)
    precision_test = precision_score(y_test, y_pred)
    recall_test = recall_score(y_test, y_pred)
    F1_score_test = f1_score(y_test, y_pred)
    # print("F1_Score = ", F1_score_test )
    roc_auc_test = roc_auc_score(y_test, y_pred_proba[:, 1])
    
    #cross-validation
    cross_val = cross_val_score(model, X_train, y_train, cv=5).mean()
    
    return (
        accuracy_train,
        precision_train,
        recall_train,
        F1_score_train,
        roc_auc_train,
        accuracy_test,
        precision_test,
        recall_test,
        F1_score_test,
        roc_auc_test,
        cross_val
    )
    
    
def apply_models_with_default_paramters(X_train, X_test, y_train, y_test):
    models_default = [
        {"ModelNames": "SVM", "Model": SVC(kernel="poly", probability=True)},
        {
            "ModelNames": "RF",
            "Model": RandomForestClassifier(n_estimators=100, random_state=42),
        },
        {"ModelNames": "KNN", "Model": KNeighborsClassifier(n_neighbors=5)},
        {"ModelNames": "XGB", "Model": XGBClassifier()},
        {"ModelNames": "LR", "Model": LogisticRegression(max_iter=1000)},
    ]
    
    cross_val_train = []

    F1_Score_train = []
    Accuracy_train = []
    Recall_train = []
    Precision_train = []
    ROC_AUC_Score_train = []

    F1_Score_test = []
    Accuracy_test = []
    Recall_test = []
    Precision_test = []
    ROC_AUC_Score_test = []

    Model_Name = []

    for model in models_default:
        # print(model)
        Model_Name.append(model["ModelNames"])
        model["Model"].fit(X_train, y_train)

        (
            accuracy_train,
            precision_train,
            recall_train,
            F1_score_train,
            roc_auc_train,
            accuracy_test,
            precision_test,
            recall_test,
            F1_score_test,
            roc_auc_test,
            cross_val            
        ) = Evaluation(model["Model"], X_train, X_test, y_train, y_test, False)

        cross_val_train.append(cross_val)
        
        F1_Score_train.append(F1_score_train)
        Accuracy_train.append(accuracy_train)
        Recall_train.append(recall_train)
        Precision_train.append(precision_train)
        ROC_AUC_Score_train.append(roc_auc_train)

        F1_Score_test.append(F1_score_test)
        Accuracy_test.append(accuracy_test)
        Recall_test.append(recall_test)
        Precision_test.append(precision_test)
        ROC_AUC_Score_test.append(roc_auc_test)

    results = pd.DataFrame()
    results["Model_Name"] = Model_Name

    train_test_f1_score_difference = np.subtract(
        F1_Score_train, F1_Score_test
    )  # To Check Overfitting/Underfitting

    results["Cross validation mean"] = cross_val_train
    
    results["Accuracy on Test Set"] = Accuracy_test
    results["Precision on Test Set"] = Precision_test
    results["Recall on Test Set"] = Recall_test
    results["F1_Score on Test Set"] = F1_Score_test         
    results["ROC_AUC_Score on Test Set"] = ROC_AUC_Score_test

    results["Accuracy on Train Set"] = Accuracy_train
    results["Precision on Train Set"] = Precision_train
    results["Recall on Train Set"] = Recall_train
    results["F1_Score on Train Set"] = F1_Score_train      
    results["ROC_AUC_Score on Train Set"] = ROC_AUC_Score_train

    results["Difference of F1_Score on train and test"] = train_test_f1_score_difference

    results = results.sort_values(
        by=["F1_Score on Test Set", "Difference of F1_Score on train and test"],
        ascending=[False, True],
    )

    return results

In [52]:
#X_train, X_test, y_train, y_test
Results_cn_supervised = apply_models_with_default_paramters(X_train, X_test, y_train, y_test)


In [53]:
Results_cn_supervised

,Model_Name,Cross validation mean,Accuracy on Test Set,Precision on Test Set,Recall on Test Set,F1_Score on Test Set,ROC_AUC_Score on Test Set,Accuracy on Train Set,Precision on Train Set,Recall on Train Set,F1_Score on Train Set,ROC_AUC_Score on Train Set,Difference of F1_Score on train and test
0,SVM,0.829375,0.8350,0.845771,0.829268,0.837438,0.913046,0.913750,0.923274,0.90250,0.912769,0.971920,0.075330
4,LR,0.826875,0.8325,0.841584,0.829268,0.835381,0.908168,0.864375,0.864831,0.86375,0.864290,0.940311,0.028909
3,XGB,0.745000,0.7850,0.787440,0.795122,0.791262,0.866191,1.000000,1.000000,1.00000,1.000000,1.000000,0.208738
1,RF,0.761250,0.7800,0.793970,0.770732,0.782178,0.849844,1.000000,1.000000,1.00000,1.000000,1.000000,0.217822
2,KNN,0.690625,0.7050,0.773585,0.600000,0.675824,0.770707,0.824375,0.883309,0.74750,0.809749,0.903359,0.133925
